# Refatorado

Nossa companhia tem uma API que conta o número de letras em um texto.

In [111]:
import abc
from itertools import cycle
from string import ascii_lowercase
from collections import Counter


class Server(abc.ABC):
    """
    Application the counts the number of letter in a piece of text.
    """
    
    valid_chars = set(ascii_lowercase)
    
    def __repr__(self):
        return self.id_
    
    def __hash__(self):
        return hash(self.id_)
    
    @property
    def cpu(self) -> int:
        """
        Number of CPUs.
        """
        return self._cpu
    
    @property
    def mem(self) -> int:
        """
        Total memory.
        """
        return self._mem
    
    @property
    def id_(self) -> str:
        """
        Server id.
        """
        return self._id
    
    def display_svr_info(self):
        print(f"ID: {self.id_}, MEM: {self.mem}, CPU: {self.cpu}")
    
    def serve(self, req: dict) -> dict:
        """
        Count the number of latter in req.
        """
        text = req["text"].lower()
        text = [char for char in text if char in self.valid_chars]
        ctr = Counter(text)
        print(self.id_, ctr)
        return ctr
    
class Worker(Server):
    """
    A single working node.
    """
    def __init__(self, id_: str, mem: int, cpu: int):
        self._mem = mem
        self._cpu = cpu
        self._id = id_
        
class Cluster(Server):
    """
    A cluster of nodes or clusters.
    """
    def __init__(self, id_: str, *servers: 'Server'):
        self._id = id_
        self._servers = set(servers)
        self._splitter = None
        
    @property
    def cpu(self) -> int:
        return sum(svr.cpu for svr in self._servers)
    
    @property
    def mem(self) -> int:
        return sum(svr.mem for svr in self._servers)
    
    def _create_splitter(self):
        """
        Creates an object that cycles the servers.
        """
        def iterator():
            for worker in cycle(self._servers):
                req = yield
                worker.serve(req)
        splitter = iterator()
        splitter.send(None)
        return splitter
    
    def serve(self, req: dict) -> dict:
        if not self._splitter:
            self._splitter = self._create_splitter()
            
        self._splitter.send(req)
        
    def add(self, *servers: 'Server'):
        self._servers |= set(servers)
        self._splitter = None
        
    def remove(self, *servers: 'Server'):
        self._servers -= set(servers)
        self._splitter = None

Vamos testar com um único worker.

In [112]:
requests = [
    {"id": 1, "text": "I'm sorry dave."},
    {"id": 2, "text": "I'm afraid I can't do that."}
]

svr = Worker("hal", 16, 6)
svr.display_svr_info()

print()
print("serving...")
for req in requests:
    svr.serve(req)

ID: hal, MEM: 16, CPU: 6

serving...
hal Counter({'r': 2, 'i': 1, 'm': 1, 's': 1, 'o': 1, 'y': 1, 'd': 1, 'a': 1, 'v': 1, 'e': 1})
hal Counter({'a': 4, 'i': 3, 't': 3, 'd': 2, 'm': 1, 'f': 1, 'r': 1, 'c': 1, 'n': 1, 'o': 1, 'h': 1})


In [113]:
workers = [Worker(f"worker{i}", i*2, i)for i in range(8)]

cluster_a = Cluster("cluster_a")
cluster_a.add(*workers[:4])
cluster_a.display_svr_info()

cluster_b = Cluster("cluster_b")
cluster_b.add(*workers[4:])
cluster_b.display_svr_info()

cluster_mega = Cluster("mega", cluster_a, cluster_b)
cluster_mega.display_svr_info()

ID: cluster_a, MEM: 12, CPU: 6
ID: cluster_b, MEM: 44, CPU: 22
ID: mega, MEM: 56, CPU: 28


In [114]:
print("serving...")
for req in requests * 10:
    cluster_mega.serve(req)

serving...
worker5 Counter({'r': 2, 'i': 1, 'm': 1, 's': 1, 'o': 1, 'y': 1, 'd': 1, 'a': 1, 'v': 1, 'e': 1})
worker2 Counter({'a': 4, 'i': 3, 't': 3, 'd': 2, 'm': 1, 'f': 1, 'r': 1, 'c': 1, 'n': 1, 'o': 1, 'h': 1})
worker6 Counter({'r': 2, 'i': 1, 'm': 1, 's': 1, 'o': 1, 'y': 1, 'd': 1, 'a': 1, 'v': 1, 'e': 1})
worker1 Counter({'a': 4, 'i': 3, 't': 3, 'd': 2, 'm': 1, 'f': 1, 'r': 1, 'c': 1, 'n': 1, 'o': 1, 'h': 1})
worker7 Counter({'r': 2, 'i': 1, 'm': 1, 's': 1, 'o': 1, 'y': 1, 'd': 1, 'a': 1, 'v': 1, 'e': 1})
worker0 Counter({'a': 4, 'i': 3, 't': 3, 'd': 2, 'm': 1, 'f': 1, 'r': 1, 'c': 1, 'n': 1, 'o': 1, 'h': 1})
worker4 Counter({'r': 2, 'i': 1, 'm': 1, 's': 1, 'o': 1, 'y': 1, 'd': 1, 'a': 1, 'v': 1, 'e': 1})
worker3 Counter({'a': 4, 'i': 3, 't': 3, 'd': 2, 'm': 1, 'f': 1, 'r': 1, 'c': 1, 'n': 1, 'o': 1, 'h': 1})
worker5 Counter({'r': 2, 'i': 1, 'm': 1, 's': 1, 'o': 1, 'y': 1, 'd': 1, 'a': 1, 'v': 1, 'e': 1})
worker2 Counter({'a': 4, 'i': 3, 't': 3, 'd': 2, 'm': 1, 'f': 1, 'r': 1, 'c

Removendo um dos clusters.

In [118]:
cluster_mega.remove(cluster_a)
cluster_mega.display_svr_info()

ID: mega, MEM: 44, CPU: 22


In [119]:
for req in requests * 10:
    cluster_mega.serve(req)

worker7 Counter({'r': 2, 'i': 1, 'm': 1, 's': 1, 'o': 1, 'y': 1, 'd': 1, 'a': 1, 'v': 1, 'e': 1})
worker4 Counter({'a': 4, 'i': 3, 't': 3, 'd': 2, 'm': 1, 'f': 1, 'r': 1, 'c': 1, 'n': 1, 'o': 1, 'h': 1})
worker5 Counter({'r': 2, 'i': 1, 'm': 1, 's': 1, 'o': 1, 'y': 1, 'd': 1, 'a': 1, 'v': 1, 'e': 1})
worker6 Counter({'a': 4, 'i': 3, 't': 3, 'd': 2, 'm': 1, 'f': 1, 'r': 1, 'c': 1, 'n': 1, 'o': 1, 'h': 1})
worker7 Counter({'r': 2, 'i': 1, 'm': 1, 's': 1, 'o': 1, 'y': 1, 'd': 1, 'a': 1, 'v': 1, 'e': 1})
worker4 Counter({'a': 4, 'i': 3, 't': 3, 'd': 2, 'm': 1, 'f': 1, 'r': 1, 'c': 1, 'n': 1, 'o': 1, 'h': 1})
worker5 Counter({'r': 2, 'i': 1, 'm': 1, 's': 1, 'o': 1, 'y': 1, 'd': 1, 'a': 1, 'v': 1, 'e': 1})
worker6 Counter({'a': 4, 'i': 3, 't': 3, 'd': 2, 'm': 1, 'f': 1, 'r': 1, 'c': 1, 'n': 1, 'o': 1, 'h': 1})
worker7 Counter({'r': 2, 'i': 1, 'm': 1, 's': 1, 'o': 1, 'y': 1, 'd': 1, 'a': 1, 'v': 1, 'e': 1})
worker4 Counter({'a': 4, 'i': 3, 't': 3, 'd': 2, 'm': 1, 'f': 1, 'r': 1, 'c': 1, 'n': 

Ele não está mais usando os workers 0 a 3.